Conteo de palabras en Apache Pig
===

* Última modificación: Mayo 16, 2021 | YouTube

Archivos de prueba
---

A continuación se generarán tres archivos de prueba para probar el sistema. Puede usar directamente comandos del sistema operativo en el Terminal y el editor de texto `pico` para crear los archivos.

In [1]:
!rm -rf /tmp/input /tmp/output
!mkdir /tmp/input /tmp/output/

In [2]:
%%writefile /tmp/input/text0.txt
Analytics is the discovery, interpretation, and communication of meaningful patterns 
in data. Especially valuable in areas rich with recorded information, analytics relies 
on the simultaneous application of statistics, computer programming and operations research 
to quantify performance.

Organizations may apply analytics to business data to describe, predict, and improve business 
performance. Specifically, areas within analytics include predictive analytics, prescriptive 
analytics, enterprise decision management, descriptive analytics, cognitive analytics, Big 
Data Analytics, retail analytics, store assortment and stock-keeping unit optimization, 
marketing optimization and marketing mix modeling, web analytics, call analytics, speech 
analytics, sales force sizing and optimization, price and promotion modeling, predictive 
science, credit risk analysis, and fraud analytics. Since analytics can require extensive 
computation (see big data), the algorithms and software used for analytics harness the most 
current methods in computer science, statistics, and mathematics.

Writing /tmp/input/text0.txt


In [3]:
%%writefile /tmp/input/text1.txt
The field of data analysis. Analytics often involves studying past historical data to 
research potential trends, to analyze the effects of certain decisions or events, or to 
evaluate the performance of a given tool or scenario. The goal of analytics is to improve 
the business by gaining knowledge which can be used to make improvements or changes.

Writing /tmp/input/text1.txt


In [4]:
%%writefile /tmp/input/text2.txt
Data analytics (DA) is the process of examining data sets in order to draw conclusions 
about the information they contain, increasingly with the aid of specialized systems 
and software. Data analytics technologies and techniques are widely used in commercial 
industries to enable organizations to make more-informed business decisions and by 
scientists and researchers to verify or disprove scientific models, theories and 
hypotheses.

Writing /tmp/input/text2.txt


Código en Apache Pig
---

**Nota.** Se usan los dos guiones `--` para comentario de una línea y `/*` ... `*/` para comentarios de varias líneas.

In [5]:
%%writefile /tmp/script.pig

-- crea la carpeta input in el HDFS
fs -mkdir input

-- copia los archivos del sistema local al HDFS
fs -put /tmp/input/ .

-- carga de datos
lines = LOAD 'input/text*.txt' AS (line:CHARARRAY);

-- genera una tabla llamada words con una palabra por registro
words = FOREACH lines GENERATE FLATTEN(TOKENIZE(line)) AS word;

-- agrupa los registros que tienen la misma palabra
grouped = GROUP words BY word;

-- genera una variable que cuenta las ocurrencias por cada grupo
wordcount = FOREACH grouped GENERATE group, COUNT(words);

-- selecciona las primeras 15 palabras
s = LIMIT wordcount 15;

-- escribe el archivo de salida 
STORE s INTO 'output';

-- copia los archivos del HDFS al sistema local
fs -get output/ /tmp/output/


Writing /tmp/script.pig


In [6]:
!pig -execute 'run /tmp/script.pig'

2022-05-16 23:30:59,559 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2022-05-16 23:31:01,033 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.textoutputformat.separator is deprecated. Instead, use mapreduce.output.textoutputformat.separator
2022-05-16 23:31:01,228 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2022-05-16 23:31:01,334 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - yarn.resourcemanager.system-metrics-publisher.enabled is deprecated. Instead, use yarn.system-metrics-publisher.enabled
2022-05-16 23:31:01,345 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.reduce.markreset.buffer.percent is deprecated. Instead, use mapreduce.reduce.markreset.buffer.percent
2022-05-16 23:31:01,347 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.output.compres

Visualización de los resultados en el HDFS
---

In [7]:
!hdfs dfs -ls output/*

-rw-r--r--   1 root supergroup          0 2022-05-16 23:31 output/_SUCCESS
-rw-r--r--   1 root supergroup         81 2022-05-16 23:31 output/part-r-00000


In [8]:
!hdfs dfs -cat output/part-r-00000

a	1
DA	1
be	1
by	2
in	5
is	3
of	8
on	1
or	5
to	12
Big	1
The	2
aid	1
and	15
are	1


Visualilzación de resultados en el sistema local
---

In [12]:
!ls -1 /tmp/output/*

_SUCCESS
part-r-00000


In [14]:
!cat /tmp/output/part-r-00000

cat: /tmp/output/part-r-00000: No such file or directory


Notas
--

**Ejecución de programas en Grunt**

Se realiza con los comandos `exec` y `run`. 

    grunt> exec script
    
    grunt> run script
    
La diferencia entre estos comandos es que `exec` ejecuta el script sin importalo a `grunt`  mientras que `run` si lo hace.

**Ejecución de Pig en la consola de comandos**

Pig es comúnmente usado en la línea de comandos para ejecutar tareas de ETL, análisis de datos y procesamiento interactivo. Para ejecutar Pig en la línea de comandos digite:

    pig -x local
    
Si no usa la opción `x -local`, Pig usará el sistema de archivos HDFS.


La opción `-e` o `-execute` permite ejecutar un comandos simple sin entrar a Grunt:

    pig -e comando
    
    

**Ejecución de comandos del sistema operativo desde Pig**

`Pig` permite la ejecución de comandos del sistema operativo; por ejemplo:

     grunt> ls
     
También es posible usar comandos del sistema HDFS; el comando `hadoop dfs -ls /` se escribiría en `hive` como

     grunt> fs -ls / ;
     

#### Supresion de información detallada

Apache Pig imprime mucha información en pantalla relacionada con su ejecución y la Hadoop. Para regular el nivel de información entregada, se puede realizar una copia al directorio actual del archivo `./conf/log4j.properties` ubicado en la carpeta de instalación de Pig. 
    
El archivo `log4j.properties` se modifica para que se impriman únicamente los mensajes de error de Pig y Hadoop. Para ello, se modifica la línea correspondiente para que quede así:

    log4j.logger.org.apache.pig=error, A
    
y se agrega la siguiente para modificar el nivel de información entregado por Hadoop:

    log4j.logger.org.apache.hadoop=error, A
    
Se invoca Pig con:

    pig -4 log4j.properties
    
para indicar que se usa el archivo ubicado en la carpeta actual.